In [ ]:
import pandas
import requests

In [ ]:

import time
query = """
query($page:Int) {
  SiteStatistics {
    anime(page:$page, perPage: 25, sort: DATE_DESC) {
      nodes {
        date
        count
      }
    }
    manga(page:$page, perPage: 25, sort: DATE_DESC) {
      nodes {
        date
        count
      }
    }
    characters(page:$page, perPage: 25, sort: DATE_DESC) {
      nodes {
        date
        count
      }
    }
    staff(page:$page, perPage: 25, sort: DATE_DESC) {
      nodes {
        date
        count
      }
    }
  }
}
"""

all_data = []
page = 1
while True:
    variables = {
        'page': page
    }
    response = requests.post('https://graphql.anilist.co', json={'query': query, 'variables': variables})
    response.raise_for_status()
    data = response.json()
    all_data.append(data)
    if len(data['data']['SiteStatistics']['anime']['nodes']) == 0:
        break
    page += 1
    time.sleep(1)

In [ ]:
import datetime

data: dict[str, dict[datetime.date, int]] = dict(anime={}, manga={}, characters={}, staff={})
for page in all_data:
    for key in data:
        for node in page['data']['SiteStatistics'][key]['nodes']:
            date = datetime.datetime.fromtimestamp(node['date']).date()
            count = node['count']
            data[key][date] = count

In [ ]:
rows: list[tuple[datetime.date, str, int]] = []
# Tabular format: (date (YYYY-MM-DD), category, count)
for key in data:
    for date, count in data[key].items():
        rows.append((date, key, count))

rows.sort()

In [ ]:
import json
json_data = [[date.isoformat(), key, count] for (date, key, count) in rows]
with open("data.json", "w", encoding="utf-8") as f:
    json.dump(json_data, f)

In [ ]:
year_month_records: dict[str, dict[tuple[int, int], tuple[datetime.date, int]]] = dict(anime={}, manga={}, characters={}, staff={})
for key in data:
    for date, count in reversed(data[key].items()):
        year_month = (date.year, date.month)
        year_month_records[key][year_month] = (date, count)

In [21]:
retrieved = datetime.date.today()
commons = "Q565"
source_url = "https://anilist.co/site-stats"
import_url = "https://commons.wikimedia.org/wiki/Data:Anilist-item-count-2023-04-02.tab"
props = {"anime": "P8729", "manga": "P8731", "staff": "P11227"}
line = '{prop}|P4876|{count}|P585|{data_date}|!S854|"{source_url}"|S813|{retrieved}|!S143|{commons}|S4656|"{import_url}"|S813|{retrieved}'

def format_date(date: datetime.date) -> str:
    return f"+{date.year:04d}-{date.month:02d}-{date.day:02d}T00:00:00Z/11"

for category, category_data in year_month_records.items():
    for (year, month), (date, count) in category_data.items():
        if category not in props:
            continue
        print(line.format(
            prop=props[category],
            count=count,
            data_date=format_date(date),
            source_url=source_url,
            retrieved=format_date(retrieved),
            commons=commons,
            import_url=import_url,
        ))

P8729|P4876|13072|P585|+2019-07-31T00:00:00Z/11|!S854|"https://anilist.co/site-stats"|S813|+2023-04-02T00:00:00Z/11|!S143|Q565|S4656|"https://commons.wikimedia.org/wiki/Data:Anilist-item-count-2023-04-02.tab"|S813|+2023-04-02T00:00:00Z/11
P8729|P4876|13116|P585|+2019-08-31T00:00:00Z/11|!S854|"https://anilist.co/site-stats"|S813|+2023-04-02T00:00:00Z/11|!S143|Q565|S4656|"https://commons.wikimedia.org/wiki/Data:Anilist-item-count-2023-04-02.tab"|S813|+2023-04-02T00:00:00Z/11
P8729|P4876|13235|P585|+2019-09-30T00:00:00Z/11|!S854|"https://anilist.co/site-stats"|S813|+2023-04-02T00:00:00Z/11|!S143|Q565|S4656|"https://commons.wikimedia.org/wiki/Data:Anilist-item-count-2023-04-02.tab"|S813|+2023-04-02T00:00:00Z/11
P8729|P4876|13303|P585|+2019-10-31T00:00:00Z/11|!S854|"https://anilist.co/site-stats"|S813|+2023-04-02T00:00:00Z/11|!S143|Q565|S4656|"https://commons.wikimedia.org/wiki/Data:Anilist-item-count-2023-04-02.tab"|S813|+2023-04-02T00:00:00Z/11
P8729|P4876|13375|P585|+2019-11-30T00:00:00Z